# Test all the QIR techniques implemented

In [ ]:
from old.dataLoader import MNISTDataLoader,CatsDataset
from old.FRQI import FRQI
from old.NEQR import NEQR
from old.OQIM import OQIM
from old.postprocessing import Postprocessing
from old.functions import calc_difference,visualize_images_grid,execute_sim

import numpy as np
from qiskit import Aer, execute
from qiskit.tools.monitor import job_monitor

### Parameter initialitzation

In [ ]:
num_pixels = [16,64]
input_images = []
dataset = 'digits'

# Set parameters the get an image
batch_size_train = 4
random_seed = 4

#set parameters for the simulation
numOfShots = 8000
method = 'OQIM'
printTime = True
sim_bool = True
#num_simulations = 1

#set backends
measure_bool = True
backend =  Aer.get_backend('qasm_simulator')


### Import and visualize the image

In [ ]:
if dataset == 'cats':
    for pixels in num_pixels:
        loader = CatsDataset('datasets/train_catvnoncat.h5')
        train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes = loader.load_and_resize_images(size=(pixels, pixels))
        #image,lable = loader.get_image(train_set_x_orig[2],train_set_y_orig[2])
        input_images.append(train_set_x_orig[2])
else:
    for pixels in num_pixels:
        data_loader = MNISTDataLoader()
        train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig = data_loader.load_and_resize_images(pixels=pixels)
        #input_image,input_label= data_loader.get_image(train_set_x_orig[1],train_set_y_orig[1])
        input_images.append(train_set_x_orig[1])

### Creation of the quantum circuit

This is the circuit that we need to convert the classical image to a quantum state

In [ ]:
# Create instances of the classes
oqim = OQIM()
neqr = NEQR()
frqi = FRQI()
circs = []
num_qubits = []

for i in range(len(num_pixels)):
    N = int(np.log2(num_pixels[i]))
    pos_qubits = 2 * N
    # Create circuits
    if method == 'FRQI':
        circuit = frqi._create_FRQI_circ(input_images[i], measure_bool=measure_bool, printTime=printTime)
        num_qubits.append(pos_qubits+1)
    elif method == 'NEQR':
        circuit = neqr._create_NEQR_circ(input_images[i], measure_bool=measure_bool, printTime=printTime)
        num_qubits.append(pos_qubits+8)
    elif method == 'OQIM':
        circuit = oqim.create_OQIM_circ(input_images[i], measure_bool=measure_bool, printTime=printTime)
        num_qubits.append(pos_qubits+2)

    #pennylaneCircuit = qml.from_qiskit(circuit)
    circs.append(circuit)

## Simulation

In [ ]:
counts = execute_sim(circs, numOfShots, backend, sim_bool)

## Image Retrival

In [ ]:
postprocessing = Postprocessing()
outputs=[]
list = []

for i in range(len(num_pixels)):
        '''dev = qml.device("qiskit.aer", wires=num_qubits[i],shots = numOfShots)

        @qml.qnode(dev)
        def circuit():
                penny_circs[i]()
                return qml.counts()
        result = circuit()'''
        if(len(num_pixels) == 1):
                image_out = postprocessing.postprocess(method,counts, input_images[i])
        else:
                image_out = postprocessing.postprocess(method,counts[i], input_images[i])
        diff=calc_difference(image_out,input_images[i])
        print('Output_image with backend and method '+method+': --> ', diff, '%')
        outputs.append(image_out)

### Original image

In [ ]:
visualize_images_grid(num_pixels,1, input_images,1)

In [ ]:
num_rows = 1
visualize_images_grid(num_pixels,num_rows, outputs,numOfShots)

## Real Machine tests

In [ ]:
num_qubits = circs[0].num_qubits

from pprint import pprint # pretty printer
from qiskit_ibm_provider import IBMProvider
from qiskit_ibm_provider import least_busy

token='0d6d7663550d808dd1c67ed5ddbe07a617fb5165f39017cf1925cfa26986513bc536f3ddd86e26086e9e49a6886c42b9ff035ef63689912ae812954248b8590a'# Insert your token here
IBMProvider.save_account(token, overwrite=True)
provider = IBMProvider()

def backend_filter_num_circuit(b):
    return b.configuration().n_qubits >= num_qubits and not b.configuration().simulator and b.status().operational==True and b.status().status_msg=='active'


backends_list=provider.backends(filters=backend_filter_num_circuit)

# display current supported backends
pprint(backends_list)

In [ ]:
# Just take the least_busy backend
backend_real=least_busy(provider.backends(filters=backend_filter_num_circuit))
print(backend_real)

##  Transpilation

To get a feeling about the complexity of the circuits, it is always advisable to track the circuit depth and number of quantum gates before running the circuit on the real backends

In [ ]:
#We need to transpile the circuit for the chosen backend
from qiskit import assemble

new_circs = []
for i in range(len(num_pixels)):
    print('Used method:', method)
    print('Number of qubits used', )
    print('Circuit_depth :', circs[i].depth())
    print('Operations :', circs[i].count_ops())

In [ ]:
print('Used backend: ', backend_real)
job = execute(circs, backend_real, shots=numOfShots)
print('Job_ID: ', job.job_id())
result = job.result()
results = result.get_counts()
if sim_bool == False:
    print(job_monitor(job))
    # timecalculation(job)
else:
    execution_time = result.time_taken # Calculate time
    print('Execution time: ', str(round(execution_time,2))+'s')    

In [ ]:
postprocessing = Postprocessing()
outputs=[]
list = []

for i in range(len(num_pixels)):
        
        if(len(num_pixels) == 1):
                image_out = postprocessing.postprocess(method,results, input_images[i])
        else:
                image_out = postprocessing.postprocess(method,results[i], input_images[i])
        diff=calc_difference(image_out,input_images[i])
        print('Output_image with backend '+backend_real.name+' and method '+method+':', image_out, ' --> ', diff, '%')
        outputs.append(image_out)

In [ ]:
num_rows = 1
visualize_images_grid(num_pixels,num_rows, outputs,numOfShots)